In [6]:
import pandas as pd
import numpy as np
from pandas import DataFrame
import warnings
warnings.filterwarnings(action="ignore")
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import binarize
from sklearn.preprocessing import LabelEncoder
from plotnine import *
import missingno as msno
import matplotlib
from matplotlib import font_manager, rc
import platform
import re

In [ ]:
# 목표 : 등수올리기
# - 조건문 개선 
# - Name => Age결측값 처리
# Mrs => Mrs 그룹의 중앙값
# Miss => Miss 그룹의 중앙값
# .......
# Mr
# 나머지는 전체 데이터의 중앙값으로
# -FamilySize 컬럼 추가 (SibSp+Parch), cut함수 써서 bin

In [115]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")

train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [116]:
test.Age

0      34.5
1      47.0
2      62.0
3      27.0
4      22.0
       ... 
413     NaN
414    39.0
415    38.5
416     NaN
417     NaN
Name: Age, Length: 418, dtype: float64

In [84]:
mrs=[]
mr=[]
miss=[]
master=[]
others=[]
cnt=0

for i in train.Name:
    pat=re.search("\w+,\s([\w+,]+)",i)
    if pat.group(1) == "Mr":
        mr.append(train.loc[cnt,"Age"])
        
    elif pat.group(1) == "Miss":
        miss.append(train.loc[cnt,"Age"])
        
    elif pat.group(1) == "Mrs":
        mrs.append(train.loc[cnt,"Age"])
    
    elif pat.group(1) == "Master":
        master.append(train.loc[cnt,"Age"])
    else:
        others.append(train.loc[cnt,"Age"])
    cnt+=1

median_mrs=(max(mrs)+min(mrs))/2
median_mr=(max(mr)+min(mr))/2
median_miss=(max(miss)+min(miss))/2
median_master=(max(master)+min(master))/2
median_others=(max(others)+min(others))/2

cnt=0
for i in train.Name:
    pat=re.search("\w+,\s([\w+,]+)",i)
    if pat.group(1) == "Mr":
        if train.loc[cnt,"Age"] == np.NaN:
            train.loc[cnt,"Age"]=median_mr
        
    elif pat.group(1) == "Miss":
        if train.loc[cnt,"Age"] == np.NaN:
            train.loc[cnt,"Age"]=median_miss
        
    elif pat.group(1) == "Mrs":
        if train.loc[cnt,"Age"] == np.NaN:
            train.loc[cnt,"Age"]=median_mrs
        
    
    elif pat.group(1) == "Master":
        if train.loc[cnt,"Age"] == np.NaN:
            train.loc[cnt,"Age"]=median_master
    else:
        train.loc[cnt,"Age"]=median_others
    cnt+=1

train.Age

0      22.0
1      38.0
2      26.0
3      35.0
4      35.0
       ... 
886    46.5
887    19.0
888     NaN
889    26.0
890    32.0
Name: Age, Length: 891, dtype: float64

In [133]:
test_mr=[]
test_mrs=[]
test_miss=[]
test_master=[]
test_others=[]
cnt=0
for i in test.Name:
    pat=re.search("\w+,\s([\w+,]+)",i)
    if pat.group(1) == "Mr":
        test_mr.append(test.loc[cnt,"Age"])
        
    elif pat.group(1) == "Miss":
        test_miss.append(test.loc[cnt,"Age"])
        
    elif pat.group(1) == "Mrs":
        test_mrs.append(test.loc[cnt,"Age"])
    
    elif pat.group(1) == "Master":
        test_master.append(test.loc[cnt,"Age"])
    else:
        test_others.append(train.loc[cnt,"Age"])
    cnt+=1

test_median_mrs=(max(test_mrs)+min(test_mrs))/2
test_median_mr=(max(test_mr)+min(test_mr))/2
test_median_miss=(max(test_miss)+min(test_miss))/2
test_median_master=(max(test_master)+min(test_master))/2
test_median_others=(max(test_others)+min(test_others))/2

cnt=0
for i in test.Name:
    pat=re.search("\w+,\s([\w+,]+)",i)
    if pat.group(1) == "Mr":
        if test.loc[cnt,"Age"] == None:
            test.loc[cnt,"Age"]=test_median_mr
        
    elif pat.group(1) == "Miss":
        if test.loc[cnt,"Age"] == None:
            test.loc[cnt,"Age"]=test_median_miss
        
    elif pat.group(1) == "Mrs":
        if test.loc[cnt,"Age"] == None:
            test.loc[cnt,"Age"]=test_median_mrs
        
    
    elif pat.group(1) == "Master":
        if test.loc[cnt,"Age"] == None:
            test.loc[cnt,"Age"]=test_median_master
    else:
        test.loc[cnt,"Age"]=test_median_others
    cnt+=1

test.Age

0      34.5
1      47.0
2      62.0
3      27.0
4      22.0
       ... 
413     NaN
414    22.5
415    38.5
416     NaN
417     NaN
Name: Age, Length: 418, dtype: float64

In [92]:
test.Age[416]

nan

In [87]:
df=pd.read_csv("train.csv")
df
title=[]

for i in df.Name:
    pat=re.search("\w+,\s([\w+,]+)",i)
    title.append(pat.group(1))

# 중복제거하려고 set 사용, 호칭을 담은 리스트와 count할때 set은 사용할 수 없어서 list로 한번 더 변환    
title_list=set(title)
#title_list
tl=list(title_list)

for i in range(len(tl)):
    print(tl[i],"호칭의 개수 :",title.count(tl[i]))

Lady 호칭의 개수 : 1
Major 호칭의 개수 : 2
Mr 호칭의 개수 : 517
Rev 호칭의 개수 : 6
Capt 호칭의 개수 : 1
Jonkheer 호칭의 개수 : 1
Dr 호칭의 개수 : 7
Mme 호칭의 개수 : 1
Ms 호칭의 개수 : 1
Master 호칭의 개수 : 40
Mrs 호칭의 개수 : 125
Mlle 호칭의 개수 : 2
Col 호칭의 개수 : 2
Sir 호칭의 개수 : 1
the 호칭의 개수 : 1
Don 호칭의 개수 : 1
Miss 호칭의 개수 : 182
